## Ввод датасета/нашей документации

In [1]:
from striprtf.striprtf import rtf_to_text

In [2]:
file=open('001.rtf', 'r')

In [3]:
rtfdoc=file.read()

In [4]:
document=rtf_to_text(rtfdoc)

In [5]:
document

'\nНаименование документа|ИНСТРУКЦИЯ "АГЕНТА КОММЕРЧЕСКОГО"|\nВид документа|ИНСТРУКЦИЯ|\nДолжностная инструкция|АГЕНТ КОММЕРЧЕСКИЙ|\nОглавление|ОБРАЗЕЦ ДОЛЖНОСТНОЙ ИНСТРУКЦИИ\n     КОММЕРЧЕСКОЕ ПОСРЕДНИЧЕСТВО\n          ТЕХНИЧЕСКИЕ ИСПОЛНИТЕЛИ|\n\n\nКомментарии\nТекст документа\n\nИНСТРУКЦИЯ\n\nАГЕНТА КОММЕРЧЕСКОГО\n\nДОЛЖНОСТНАЯ ИНСТРУКЦИЯ|УТВЕРЖДАЮ|\n||\n|(орган юридического лица (учредители);|\n00.00.00  № 00|лицо, уполномоченное утверждать|\nагента коммерческого|должностную инструкцию)|\n||||\n|(подпись)||(фамилия, инициалы)|\n|00.00.00 г.|||\n\n\nI. Общие положения\n\n1. Коммерческий агент относится к категории технических исполнителей.\n2. На должность коммерческого агента назначается лицо, имеющее среднее профессиональное образование и специальная подготовка по установленной программе без предъявления требований к стажу работы.\n3. Назначение на должность коммерческого агента и освобождение от нее производится приказом директора предприятия.\n4. Коммерческий агент должен знать:\

## Готовим инструменты

In [6]:
import re
import logging

import gensim.downloader as api
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

import pymorphy2

#from nltk import pos_tag_sents
#from nltk.tokenize import word_tokenize
#nltk.download('punkt')
#nltk_download('averaged_perceptron_tagger_ru')
#nltk.download('universal_tagset')
#pos_tag_sents(word_tokenize("настоящей должностной инструкцией"), tagset='universal',lang='rus')

## Подключаем gensim и загружаем векторную модель слов (уже готовую, model.model)

In [13]:
import gensim

In [7]:
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [9]:
api.BASE_DIR='C:\\Users\\Home\\gensim-data\\'

In [10]:
api.BASE_DIR

'C:\\Users\\Home\\gensim-data\\'

In [11]:
#model=api.load("word2vec-ruscorpora-300")

In [15]:
md=gensim.models.KeyedVectors.load("C:\\Users\\Home\\gensim-data\\model.model") #C:\\Users\\Home\\gensim-data\\model.model

INFO - 13:58:52: loading Word2VecKeyedVectors object from C:\Users\Home\gensim-data\model.model
INFO - 13:58:52: loading vectors from C:\Users\Home\gensim-data\model.model.vectors.npy with mmap=None
INFO - 13:58:53: loading vectors_ngrams from C:\Users\Home\gensim-data\model.model.vectors_ngrams.npy with mmap=None
INFO - 13:58:59: loading vectors_vocab from C:\Users\Home\gensim-data\model.model.vectors_vocab.npy with mmap=None
INFO - 13:58:59: setting ignored attribute vectors_ngrams_norm to None
INFO - 13:58:59: setting ignored attribute vectors_norm to None
INFO - 13:58:59: setting ignored attribute buckets_word to None
INFO - 13:58:59: setting ignored attribute vectors_vocab_norm to None
INFO - 13:58:59: loaded C:\Users\Home\gensim-data\model.model


In [21]:
md.most_similar(positive=['главный', 'бухгалтер'], topn=15)

[('бухгалтерский', 0.7341619729995728),
 ('главноуправляющий', 0.697599470615387),
 ('-главный', 0.6931504011154175),
 ('.главный', 0.6719357371330261),
 ('неглавный', 0.6708090305328369),
 ('главбух', 0.6618990302085876),
 ('бухгалтерско', 0.6565037965774536),
 (',главный', 0.6553462743759155),
 ('главбухша', 0.6491634845733643),
 ('менеджерский', 0.6470004916191101),
 ('бухгалтерия', 0.6442776918411255),
 ('-администратор', 0.6385512351989746),
 ('наиглавный', 0.6352169513702393),
 ('офис-менеджер', 0.6255849599838257),
 ('глав-ный', 0.6252533197402954)]

## Получаем входной текст и предобрабатываем его: леммтизация, токенизация, выкидывание стоп-слов, частых/редких слов

In [46]:
init_text="Продажи и покупки, оформление документов, оформление договоров."

In [42]:
stop_words = stopwords.words('russian')

In [47]:
init_text=init_text.lower()

### Разбиваем текст на отдельные токены (здесь, по сути, на слова)

In [ ]:
tokenizer = RegexpTokenizer('\w+|[^\w\s]+\.') # с паттернами сложно, надо отрегулировать #'\w+|[^\w\s]+'

In [48]:
tokens=tokenizer.tokenize(init_text)

In [51]:
tokens

['продажи',
 'и',
 'покупки',
 'оформление',
 'документов',
 'оформление',
 'договоров']

### Выкидываем слова, не несущие смысловой нагрузки ("стоп-слова")

In [52]:
tokens_clear=[]
for t in tokens:
    if t in stop_words:
        continue
    tokens_clear.append(t)

In [53]:
tokens_clear

['продажи', 'покупки', 'оформление', 'документов', 'оформление', 'договоров']

### Лемматизируем (берем начальную форму)

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [54]:
tokens_lemmatized=[]
for t in tokens_clear:
    p = morph.parse(t)[0]
    p=p.normal_form
    tokens_lemmatized.append(p)

In [55]:
tokens_lemmatized

['продажа', 'покупка', 'оформление', 'документ', 'оформление', 'договор']

### Строго говоря, есть подводные камни (например, "оформление" - > "оформлять"?), но это уже тонкая настройка

### Получили обработанный вводный текст

### Оформим в виде функции

In [56]:
def prepare_text(init_text, tokenizer=tokenizer, stop_words=stop_words, morpher=morph):
    init_text=init_text.lower()
    tokens=tokenizer.tokenize(init_text)
    tokens_clear=[]
    for t in tokens:
        if t in stop_words:
            continue
        tokens_clear.append(t)
    tokens_lemmatized=[]
    for t in tokens_clear:
        p = morph.parse(t)[0]
        p=p.normal_form
        tokens_lemmatized.append(p)
    return tokens_lemmatized

## Далее следует ввод данных из датасета. Пока это символические несколько параграфов ("коммерческий агент"-2, "диспетчер", "зуботехник", "генеральный директор"-2, "заведующий ветклиникой", "заведующий рынком")

In [58]:
data_text=["1. Участвует в работе по установлению необходимых деловых контактов между покупателями и продавцами товаров, включая техническую и другую продукцию (оборудование, сырье, полуфабрикаты и др.), а также оказанию различных коммерческих услуг.",
          "2. Содействует покупке и продаже партий товаров (оптом), а также театральной, музыкальной и других видов творческой продукции (постановка представлений, выступление артистов, спортсменов, издание книг, запись музыкальных произведений и др.).",
          "4.13. Основы конструкции современной зуботехнической аппаратуры для обжига фарфора, муфельных печей, литейной аппаратуры, устройств ультразвуковых и для светоотверждения и др..",
          "2. Принимает меры по максимальному использованию производственных мощностей предприятия, содействуя рациональной загрузке оборудования, повышению коэффициента сменности, созданию условий для эффективной работы персонала.",
          "4.Организует производственно-хозяйственную деятельность на основе широкого использования новейшей техники и технологии, прогрессивных форм управления и организации труда, научно-обоснованных нормативов материальных, финансовых и трудовых затрат, изучения конъюнктуры рынка и передового опыта (отечественного и зарубежного) в целях всемерного повышения технического уровня и качества продукции (услуг), экономической эффективности ее производства, рационального использования производственных резервов и экономного расходования всех видов ресурсов.",
          "2.Организует работу и эффективное взаимодействие всех структурных подразделений, цехов и производственных единиц, направляет их деятельность на развитие и совершенствование производства с учетом социальных и рыночных приоритетов, повышение эффективности работы предприятия, рост объемов сбыта продукции и увеличение прибыли, качества и конкурентноспособности производимой продукции, ее соответствие мировым стандартам в целях завоевания отечественного и зарубежного рынка и удовлетворения потребностей населения в соответствующих видах отечественной продукции.",
          "3. Доводит до сведения потребителей информацию об оказываемых ветеринарных услугах (их перечень и формы предоставления; прейскуранты на ветеринарные услуги, утвержденные органом исполнительной власти субъекта Российской Федерации, с указанием номера и даты утверждения; лицензию; образцы препаратов, лекарственных средств и др.; средства ветеринарного назначения, применяемые при оказании платных ветеринарных услуг; нормативные документы по вопросам ветеринарного обслуживания животных; пр.).",
          "6. Обеспечивает наличие на рынке указателей торговых зон по специализации; стендов с указанием схемы размещения торговых зон, предприятий общественного питания и сервисного обслуживания, административных помещений, автостоянки, складских помещений, лабораторий и других объектов; информационного стенда, на котором вывешиваются основные нормативно-правовые, нормативно-технические документы, регламентирующие порядок осуществления торговой деятельности на рынке.",
          ]

## На этом этапе нам нужна автоматическая разбивка текста на параграфы, ну и, в целом, разметка

In [59]:
len(data_text)

8

## Итерируемся по параграфам нашего датасета и сравниваем их сс нашим входным текстом

### Каждый фрагмент (параграф) подвергнем предобработке

In [64]:
doc=prepare_text(data_text[0])

In [65]:
doc

['1',
 'участвовать',
 'работа',
 'установление',
 'необходимый',
 'деловой',
 'контакт',
 'покупатель',
 'продавец',
 'товар',
 'включая',
 'технический',
 'другой',
 'продукция',
 'оборудование',
 'сырьё',
 'полуфабрикат',
 'др',
 'также',
 'оказание',
 'различный',
 'коммерческий',
 'услуга']

### Сравниваем с нашим входным текстом, и используя метрику сравнения, выясняем, какой параграф подходит больше. Или для какого из них метрика выше порога (достаточно похож на то, что просит пользователь, чтобы показать пользователю это фрагмент)

In [66]:
# Вообще говоря, еще можно обогатить текст биграммами, и использовать это тоже

In [ ]:
# Не факт, что здесь стоит выкидывать слишком частые слова; это документация, и если слово "продажа", наример, встречается очень часто, возможно, в этом есть смысл

In [84]:
md.wmdistance(tokens_lemmatized, doc)

INFO - 16:53:08: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 16:53:08: built Dictionary(28 unique tokens: ['договор', 'документ', 'оформление', 'покупка', 'продажа']...) from 2 documents (total 29 corpus positions)


36.19322080863062

In [83]:
md.n_similarity(tokens_lemmatized, doc)

0.75448143

In [70]:
# здесь нужен pyemd.... установил

In [69]:
similarity

36.19322080863062

In [85]:
doc1=prepare_text(data_text[7])

In [78]:
md.wmdistance(tokens_lemmatized, doc1)

INFO - 16:42:49: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 16:42:49: built Dictionary(44 unique tokens: ['договор', 'документ', 'оформление', 'покупка', 'продажа']...) from 2 documents (total 53 corpus positions)


38.05926513326128

In [86]:
md.n_similarity(tokens_lemmatized, doc1)

0.7142247

In [103]:
similarities=[]
indicies=[]
#rates=dict()
sim_ind=[]
cnt=0
for text in data_text:
    doc=prepare_text(text)
    sim=md.n_similarity(tokens_lemmatized, doc)
    #rates[sim]=cnt
    sim_ind.append((sim,cnt))
    similarities.append(sim)
    indicies.append(cnt)
    cnt+=1

In [112]:
sim_ind.sort(reverse=True)

In [113]:
sim_ind

[(0.75448143, 0),
 (0.72706914, 6),
 (0.7142247, 7),
 (0.7095247, 1),
 (0.6640805, 5),
 (0.63561535, 4),
 (0.6314969, 3),
 (0.44560254, 2)]

In [116]:
n=3
top_docs=[]
for i in range(0,n):
    top_docs.append(data_text[i])

In [117]:
top_docs

['1. Участвует в работе по установлению необходимых деловых контактов между покупателями и продавцами товаров, включая техническую и другую продукцию (оборудование, сырье, полуфабрикаты и др.), а также оказанию различных коммерческих услуг.',
 '2. Содействует покупке и продаже партий товаров (оптом), а также театральной, музыкальной и других видов творческой продукции (постановка представлений, выступление артистов, спортсменов, издание книг, запись музыкальных произведений и др.).',
 '4.13. Основы конструкции современной зуботехнической аппаратуры для обжига фарфора, муфельных печей, литейной аппаратуры, устройств ультразвуковых и для светоотверждения и др..']

### Оформим это в функцию

In [120]:
def get_top_docs(init_text, data_text, model=md, n=3):
    tokens_lemmatized=prepare_text(init_text)
    sim_ind=[]
    cnt=0
    for text in data_text:
        doc=prepare_text(text)
        sim=md.n_similarity(tokens_lemmatized, doc)
        sim_ind.append((sim,cnt))
        cnt+=1
    sim_ind.sort(reverse=True)
    top_docs=[]
    for i in range(0,n):
        top_docs.append(data_text[i])
    return top_docs

In [128]:
docs=get_top_docs("взаимодействие с клиентами, оформление договоров, расчет затрат", data_text)

In [129]:
docs

['1. Участвует в работе по установлению необходимых деловых контактов между покупателями и продавцами товаров, включая техническую и другую продукцию (оборудование, сырье, полуфабрикаты и др.), а также оказанию различных коммерческих услуг.',
 '2. Содействует покупке и продаже партий товаров (оптом), а также театральной, музыкальной и других видов творческой продукции (постановка представлений, выступление артистов, спортсменов, издание книг, запись музыкальных произведений и др.).',
 '4.13. Основы конструкции современной зуботехнической аппаратуры для обжига фарфора, муфельных печей, литейной аппаратуры, устройств ультразвуковых и для светоотверждения и др..']

## Резюме: - удалять цифры из начала абзаца; - сохранять заголовки к должностным инструкциям (можно использовать для оценки качества модели)